# Datenakquise

In [8]:
# Import benötigter Module
from DDBhelpers import *
import sqlite3
import requests
from pathlib import Path
import os

## Technische Voraussetzungen

### API-Key

Für die DDB-REST-API wird ein API-Key benötigt, der [hier](https://www.deutsche-digitale-bibliothek.de/user/apikey

In [9]:
# API-Key einfügen
api_key = "<API-KEY>"

# alternativ individuellen API-Key aus Datei apikey.txt einlesen\n",
if api_key == "<API-KEY>":
    with open('apikey.txt') as IN:
        api_key = IN.read().split('\n')[0].strip()

## Datenakquise

### Überblick über Berliner Museen

Um die Datenmenge auf Objekte Berliner Museen zu beschränken, wird zunächst eine Liste der Institutions-IDs benötigt.

In [5]:
endpoint = "https://api.deutsche-digitale-bibliothek.de/search/organization"
params = {
    'query': '*',
    'facet': ['sector_fct', 'city_de_fct'],
    'rows': 1000,
    'sector_fct': 'sec_06',
    'city_de_fct': 'Berlin',
    'oauth_consumer_key': api_key
}

In [6]:
res = requests.get(endpoint, params = params)
orgas = res.json().get('results')[0].get('docs')
BerlinOrgaIDs = [_.get('id') for _ in orgas]

In [7]:
print(f"Es gibt {len(BerlinOrgaIDs)} Berliner Museen in der DDB.")

Es gibt 68 Berliner Museen in der DDB.


Wenn möglich, sollten Daten über die API geladen werden. Daher wird überprüft, welche der Institutionen ihre Daten in Gänze über die API ausspielt.

In [8]:
BerlinOrgas = dict()
for orga in BerlinOrgaIDs:
    BerlinOrgas[orga] = {
        'api' : resultNumberFromProvider(orga),
        'web' : numberFromSite(orga)
    }

### Verzeichnis aller Objekte aus Berliner Museen

In einem nächsten Schritt sollen für jede Institution die Objekt-IDs gesammelt werden.
Da es sich um viele Objekte handelt, werden sie in eine SQLITE-Datenbank (`berlinerkultuerbedaten.db`) geschrieben.

Die Datenbank enthält eine Tabelle (`objects`) mit folgenden Spalten:


| Column  | Typ | Beschreibung
|:---------|:---------|:------------|
| OBJID   | Primary Key, VARCHAR(32) | DDB-Objekt-ID (32-stellig) | 
| INSTID  | VARCHAR(32) | DDB-Organisations-ID (32-stellig) |
| ACCESS  | VARCHAR(3) | auf welchem Weg erfolgt der Datenzugriff: `WEB` oder `API`; temporär `ERR` bei Fehler im Download  |

In [10]:
# Vorbereiten der Datenbank
con = sqlite3.connect("berlinerkultuerbedaten.db")
with con:
    con.execute("""CREATE TABLE IF NOT EXISTS objects
    (
    OBJID VARCHAR(32) PRIMARY KEY,
    INSTID VARCHAR(32),
    ACCESS VARCHAR(3)
    );""")

In [13]:
for instid, objcount in reversed(BerlinOrgas.items()):
    if instid not in instIDsInDB:
        if objcount.get('api') >= objcount.get('web') and objcount.get('api') > 0:
            access = "API"
            objectIDs = iterationProvider(instid)
            rows = [(o, instid, access) for o in objectIDs]
            with con:
                con.executemany("""INSERT OR REPLACE INTO objects VALUES (?,?,?)""", rows)
        else:
            access = "WEB"
            objectIDs = iterSite(instid)
            rows = [(o, instid, access) for o in objectIDs]
            with con:
                con.executemany("""INSERT OR REPLACE INTO objects VALUES (?,?,?)""", rows)
        print(f"Für Inst. {instid} {len(rows)} Objekt-IDs via {access} ermittelt")

TTIJUOMSU2GXGIC627J2DBPZ5UWXX3UD
0
in Datenbank geschrieben
PQXATUOXS4VB7ZUB4FSK4SZ3PJKV23GE
1274
in Datenbank geschrieben
FKBVHZEFB6RYNEMU3RNQGTMCFOSV7463
634334
in Datenbank geschrieben
3NXCWZXJRWBNE2RJYONI7MMGXORZ5LK3
18289
in Datenbank geschrieben
DK57QVFKHI5RWCOBXD5YJOW2Q5EAHICS
0
in Datenbank geschrieben
PBSWRR3P3KGQYLQLRHUJTMI5BL6LRCA3
22808
in Datenbank geschrieben
API
API
API
M5IBELWI66QSLMSS7IA7WFIY7P44AR6S
2318
in Datenbank geschrieben
63GDHF5CGFG2NLODGHVZBGR5BFVTI7ZK
5431
in Datenbank geschrieben
API
ASEZP62S5KE5K2U346MJCC5EMD4PERZS
0
in Datenbank geschrieben
API
API
AO4WVRPWITXU2XHK4HDNW6OYCUIWENYS
0
in Datenbank geschrieben
3KXQ5XC4EJ73TJ4TNOZ73IOYV27X4XXC
1759
in Datenbank geschrieben
HYKWL4B74FU4J6JNZISSHIMVVUAT7YTH
2624
in Datenbank geschrieben
SKTM7G4OZSY6J7IY5JVDGCSSYRJX4EA5
0
in Datenbank geschrieben
2K4WMDXUXONP6IREIVSYTUJP6RDBZIYB
0
in Datenbank geschrieben
TIJBZTQ7GR3GXYXNY4OWHQA5S5I3OGCG
0
in Datenbank geschrieben
3T2V6C62PFNMSHP3OHFECQMYLLGCUDCK
50345
in Datenb

### Download der Daten


__ORDNERSTRUKTUR WEGEN KOMPLEXER HIERARCHISIERUNG DER INSTITUTIONEN NICHT ZWECKMÄßIG. Daten sind mehreren Institutionen zugeordnet__

Die Objekt-IDs und der privilegierte Zugriffsmodus (per WEB oder API) sind nun bekannt und in der Datenbank dokumentiert.
Mit Hilfe dieser Informationen werden nun die Datensätze aus der DDB geladen.

Die Dateien werden im Verzeichnis `data` gespeichert.
Sie werden der Übersichtlichkeit halber nach den datengebenden Institutionen in Order gelegt.
Der Dateiname entspricht dabei der Objekt-ID, der Ordnername der Organisations-ID.

Die Ordnerstruktur sieht also folgendermaßen aus (`inst_##` und `object_#` sind dabei jeweils durch DDB-IDs ersetzt):


```
data
├── inst_01
│   ├── object_a
│   ├── object_b
│   ├── object_c
└── inst_02
    ├── object_a
    ├── object_b
    ├── object_c

```

Der komplette Download der Daten nimmt viel Zeit in Anspruch.

Auf Zenodo habe ich all die Metadaten abgelegt, deren Lizenzierung die Weitergabe erlaubt. Die Datei kann heruntergeladen und im Projektordner entpackt werden. Der Ordner `data` muss dabei im selben Ordner wie die Notebooks liegen.

Die Downloadanweisung unten ergänzt lediglich nicht vorhandene Dateien.
Zum Überschreiben aller Daten `overwrite`-Flag auf `True` setzen.

In [14]:
def getSourceViaWeb(objectID) -> str:
    '''Ruft über die DDB-Objekt-ID ein Objekt der DDB auf,
    extrahiert aus den Metadaten den Quell-Datensatz und
    gibt diesen als String zurück.'''
    res = requests.get(f"https://www.deutsche-digitale-bibliothek.de/item/xml/{objectID}", timeout = 20)
    tree = etree.fromstring(res.content)
    NSMAP = {
            'source' : "http://www.deutsche-digitale-bibliothek.de/ns/cortex-item-source"
            }
    source = tree.find('.//source:source/source:record', NSMAP)
    return source.text


def getSourceViaAPI(objectID) -> str:
    '''Ruft über die DDB-Objekt-ID über die API ein Objekt der DDB auf, extrahiert den Quell-Datensatz und
    gibt diesen als String zurück.'''
    res = requests.get(f"https://api.deutsche-digitale-bibliothek.de/items/{objectID}?oauth_consumer_key={api_key}")
    source = (
        res
        .json()
        .get('source')
        .get('record')
        .get('$')
    )
    return source

In [15]:
# Download
overwrite = False
download = False

with con:
    c = con.execute('''SELECT * FROM objects''')
    for row in c:
        objID, instID, access = row
        path = Path('/home/alex/Documents/digiS/DDBdashboard/data') / instID
        Path(path).mkdir(parents = True, exist_ok = True)
        path = Path(path) / objID
        if overwrite == True:
            download = True
        if not overwrite and not path.is_file() or os.stat(path) == 0:
            download = True
        if download == True:
            with open(path, "w") as OUT:
                if access == "API":
                    source = getSourceViaAPI(objID)
                else:
                    source = getSourceViaWeb(objID)
                OUT.write(source)
                print(f"{path} geschrieben.")

/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/7VGKPB43UUS2D25ALVCQVEQGBDHTRUXV geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/VCTSWISADVDEQ2MCF66EEFYO4JBS6HX5 geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/5IR6JUFNHFR4HHDIWZZTOSLF427O7TAX geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/M5UXBRDCANNUG3CNHFW3IBXPVJ2KEARV geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/NGXVCF24PGDFXYXXDHEF2YHWPBXPGMLM geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/LDP63OFG6J744T2G2DT5AUP3CXBXFB6I geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/S6QUEVKS5O4OBCU4OJLHKYIP5HKY5RQT geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/Q43MFDQUUS73XJ3CCCXUHADLF47AMA37 geschrieben.
/home/al

/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/HX36II2ARXXIZTE7FLGQDWO5TRJYITQE geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/SDEHBF4F4IFBFGXKIWG554ZDVOXB6SHQ geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/WGJHTCVSJFAY6E64WKXG3SB6EYOOJOB2 geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/EWOAUJNPWNVDS46RDEESQ2IO254AQRL5 geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/NM3LWC2AFQQAV4ULYZP443ZZU4KNZ72J geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/XF7TAJMO2NHIAABHMJCUHJM23UMYSU7D geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/ENQUEVVNNX4BXXBG5DXSRS2SGTHDIX4W geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/YLTEAXYHB3MHMBQFAGVE362MWVNVNOOG geschrieben.
/home/al

/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/EULQYKOZXTY3J5TYZKASCQQB6I4DL7JY geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/E7WBGX6M6OJWEJUXANZXURH5LURR4JKH geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/ZH3YEHR4U6FQHVJTIRQT3AIDLPEXVJRD geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/PDF6RFGWBY6RMDVQTHMWVTLSTLRAZ5XW geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/QXTFA3UYSCYF7WQWDPAQRYVT2YKSYEBL geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/2AS3GFQSTGUY6RTLT62FSJEE5EMQKPSX geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/CDK2AWFWJ24SSPJ43ZHNSCXWRAPRFKCK geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/RYZACUCD3XV6RSG6DDAN5FCWNJ2WAEKQ geschrieben.
/home/al

/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/R7DJFHCOJ4GOD4NDWTO6A4TVM2GQSHIW geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/WPDKEG3ZONKM2EXO7FDW5RDXXDASR3SU geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/C3ZCHXCRKWKR22IAVK52HYLCRMI4VK5E geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/IXRWZXM3JCTTCKYURVJDMZZ4XUAJT6VI geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/H67656OPPLQSUGQ6A22B3H4YP3UGIYDF geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/6GPGIOFPLOR5SLJHIA2MZRJVTWEMF4X5 geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/P54DSUCYH56DGAQJEX3ULSCEH5CKQPIV geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/Y7YU6HL6HVNB42EUWTFFK2TK63RRSRSD geschrieben.
/home/al

/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/OZZLDCTAJGREMR3ZSBEBAG64YZX3AYDY geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/ILUGRVLY3JELMPJZVPQPUSKPF6A7FPSG geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/CEW5LXTVJ72WDXQQOA65U6MFS6OFLNLX geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/FHSRFJ6KLNSRNDSGPRWZXQB6HQ6ITDKD geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/D3YJYNSNSGHCPAXHB4WWJCJHFIKR3UDY geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/GWUMRH752ZUTBMI7CM2APQUZ7AUW6PYV geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/6YXQ66OI65XCEEPVMP47SIHMWWLF5236 geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/RUML2V7UOHDG2AU47VI6EZHN2O7XOYEP geschrieben.
/home/al

...

/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/K3CPP7BBWLYRR6NSOFO4BQD7WFD3QNRC geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/4GTJK5GCEPT3LCTSI6CUBKYKBP3A6MFQ geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/Z6N7TYKKH6MM4TR3HHXCLNYNWG47V63P geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/O7KQZXKNTAB5JQZQNGZ2ERYAFZ2ZDQZR geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/7D6AYCINGBXAVOOWNNKGUD7MWGRC4RAV geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/FDA565DCJAUXSDWOMGOSXUKNFXVY4GBD geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/FA5YZUHDXWNTQUFDTMV7IKZ4VHRG63WQ geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/B3IF7HHZWY2M5O6JCILCV4UYYZFX3TKT geschrieben.
/home/al

/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/IKKNVLQLZUJQP2AMPSY6QKGNNQ23B6PK geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/QPE7QNABHPCOIR3Y4N467XYOJHZFCCEG geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/3CP3O7SDTB554TOOXMPE4UNFVSK4XZ5V geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/4WR4QGR7VPFN77TNWVMY7DXZCAQIEMH4 geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/KIGDTMPQH6OH3KPX5WS4AQAYUOEOANM7 geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/QG34AQK7GVE7OJLWG4DR6FR2JJQLHE4B geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/H46PWY3MDSEJQX3V62BF57FHVZ5VT3BY geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/XBOOWEE5EK33N74KECSJ4TZ6WLDT36KR geschrieben.
/home/al

/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/Y4KALOHC6SFP4SI27RT7HNI4AVOJ2YKJ geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/SU6KM2LO5EGS4AHJFXTHGI6Q6LBRI542 geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/YHK6IOKQ6JZQ4SFYQDZC7OX6WSJPX4BH geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/3HMAMLGOSRO5ZSCBTU627IGVAT2OS6IV geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/RY3XBY4UHNV27FTKBNCP2E7E3E3AWBZR geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/3YQZPFL73UTTCJAFRUJD3HHMZPRWTP2D geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/PWGYS3WRWKE5ENJADORA3NVX73HQUK4S geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/TRROKMU7JHIFT6T7NYBGM55LCFLZNVI5 geschrieben.
/home/al

/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/J7XJKL3LXD5YYTTGOU2NNE3B2AIYCQAN geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/DKY2P54G7JLCYIC42NWU6GFZKQO662PI geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/O22Q3H6EGV3GQVDAU3PNSJ5F2KRAR2C2 geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/VYEYQA3UJXJGE7XDPMZ3PPX4RJKKMOST geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/NPGVJVCNYQNZBQD3VKV4QXIYCF5KWRFE geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/YPP5LQHT67KJBJYSQ4GDRY3TGCN4B55V geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/H256O4YOY4LSHMVEMOOOKR6AFUBNZ4DL geschrieben.
/home/alex/Documents/digiS/DDBdashboard/data/VH3B3BVEVLGX2XZA5SRAL2CYJOW6YV76/EQBR6RJ2IA3IC4PKGIUVC4UKQ6KTEZP6 geschrieben.
/home/al

KeyboardInterrupt: 